# L4 pygame快速入門&範例

In [5]:
!pip install pygame

In [2]:
import pygame

pygame 2.1.2 (SDL 2.0.18, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


### 入門範例- pygame的基礎模板

In [5]:
import pygame # 引入 pygame 模組
import sys

pygame.init() # 初始化 pygame 模組

#螢幕大小
window = pygame.display.set_mode((800,450))  
#螢幕標題
pygame.display.set_caption("這是一隻 PyGame 的範例程式")
#螢幕顏色
window.fill((255, 255, 255))

pygame.display.flip() #將上方設定好的視窗 顯示出來
 
# 遊戲迴圈 在發生 if e.type == pygame.QUIT: 之前都會一直執行
while True:
	# 捕捉 pygame 接受到的事件，並且針對關閉視窗的事件做處理
    for e in pygame.event.get():
        if e.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
            
        #遊戲的內容從這邊開始
           

SystemExit: 

### 舉例1- 操控移動

In [2]:
import pygame
import sys

# 初始化pygame
pygame.init()
# 定義變數
size = width, height = 600, 400
bg = (255, 255, 255)

# 加載圖片
img = pygame.image.load("./ball.jpg")
# 獲取圖片位置
position = img.get_rect()
# 創建主視窗
screen = pygame.display.set_mode(size)
# 設定視窗標題
pygame.display.set_caption("Pygame練習")
# 創建遊戲主迴圈
while True:
    # 設定初始值
    site = [0, 0]
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        # 圖片移動，透過偵測鍵盤按鍵事件實現
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_UP:
                site[1] -= 8
            if event.key == pygame.K_DOWN:
                site[1] += 8
            if event.key == pygame.K_LEFT:
                site[0] -= 8
            if event.key == pygame.K_RIGHT:
                site[0] += 8
    # 移動圖片
    position = position.move(site)
    # 填充背景
    screen.fill(bg)
    # 放置圖片
    screen.blit(img, position)
    # 更新顯示界面
    pygame.display.flip()
    


SystemExit: 

### 範例 對對碰
來源: https://inventwithpython.com/pygame/chapter3.html

In [1]:
# Memory Puzzle
# By Al Sweigart al@inventwithpython.com
# http://inventwithpython.com/pygame
# Released under a "Simplified BSD" license

import random, pygame, sys
from pygame.locals import *

FPS = 30 # frames per second, the general speed of the program
WINDOWWIDTH = 640 # size of window's width in pixels
WINDOWHEIGHT = 480 # size of windows' height in pixels
REVEALSPEED = 8 # speed boxes' sliding reveals and covers
BOXSIZE = 40 # size of box height & width in pixels
GAPSIZE = 10 # size of gap between boxes in pixels
BOARDWIDTH = 5 # number of columns of icons
BOARDHEIGHT = 4 # number of rows of icons
assert (BOARDWIDTH * BOARDHEIGHT) % 2 == 0, 'Board needs to have an even number of boxes for pairs of matches.'
XMARGIN = int((WINDOWWIDTH - (BOARDWIDTH * (BOXSIZE + GAPSIZE))) / 2)
YMARGIN = int((WINDOWHEIGHT - (BOARDHEIGHT * (BOXSIZE + GAPSIZE))) / 2)

#            R    G    B
GRAY     = (100, 100, 100)
NAVYBLUE = ( 60,  60, 100)
WHITE    = (255, 255, 255)
RED      = (255,   0,   0)
GREEN    = (  0, 255,   0)
BLUE     = (  0,   0, 255)
YELLOW   = (255, 255,   0)
ORANGE   = (255, 128,   0)
PURPLE   = (255,   0, 255)
CYAN     = (  0, 255, 255)

BGCOLOR = NAVYBLUE
LIGHTBGCOLOR = GRAY
BOXCOLOR = WHITE
HIGHLIGHTCOLOR = BLUE

DONUT = 'donut'
SQUARE = 'square'
DIAMOND = 'diamond'
LINES = 'lines'
OVAL = 'oval'

ALLCOLORS = (RED, GREEN, BLUE, YELLOW, ORANGE, PURPLE, CYAN)
ALLSHAPES = (DONUT, SQUARE, DIAMOND, LINES, OVAL)
assert len(ALLCOLORS) * len(ALLSHAPES) * 2 >= BOARDWIDTH * BOARDHEIGHT, "Board is too big for the number of shapes/colors defined."

def main():
    global FPSCLOCK, DISPLAYSURF
    pygame.init()
    FPSCLOCK = pygame.time.Clock()
    DISPLAYSURF = pygame.display.set_mode((WINDOWWIDTH, WINDOWHEIGHT))

    mousex = 0 # used to store x coordinate of mouse event
    mousey = 0 # used to store y coordinate of mouse event
    pygame.display.set_caption('對對碰')

    mainBoard = getRandomizedBoard()
    revealedBoxes = generateRevealedBoxesData(False)

    firstSelection = None # stores the (x, y) of the first box clicked.

    DISPLAYSURF.fill(BGCOLOR)
    startGameAnimation(mainBoard)

    while True: # main game loop
        mouseClicked = False

        DISPLAYSURF.fill(BGCOLOR) # drawing the window
        drawBoard(mainBoard, revealedBoxes)

        for event in pygame.event.get(): # event handling loop
            if event.type == QUIT or (event.type == KEYUP and event.key == K_ESCAPE):
                pygame.quit()
                sys.exit()
            elif event.type == MOUSEMOTION:
                mousex, mousey = event.pos
            elif event.type == MOUSEBUTTONUP:
                mousex, mousey = event.pos
                mouseClicked = True

        boxx, boxy = getBoxAtPixel(mousex, mousey)
        if boxx != None and boxy != None:
            # The mouse is currently over a box.
            if not revealedBoxes[boxx][boxy]:
                drawHighlightBox(boxx, boxy)
            if not revealedBoxes[boxx][boxy] and mouseClicked:
                revealBoxesAnimation(mainBoard, [(boxx, boxy)])
                revealedBoxes[boxx][boxy] = True # set the box as "revealed"
                if firstSelection == None: # the current box was the first box clicked
                    firstSelection = (boxx, boxy)
                else: # the current box was the second box clicked
                    # Check if there is a match between the two icons.
                    icon1shape, icon1color = getShapeAndColor(mainBoard, firstSelection[0], firstSelection[1])
                    icon2shape, icon2color = getShapeAndColor(mainBoard, boxx, boxy)

                    if icon1shape != icon2shape or icon1color != icon2color:
                        # Icons don't match. Re-cover up both selections.
                        pygame.time.wait(1000) # 1000 milliseconds = 1 sec
                        coverBoxesAnimation(mainBoard, [(firstSelection[0], firstSelection[1]), (boxx, boxy)])
                        revealedBoxes[firstSelection[0]][firstSelection[1]] = False
                        revealedBoxes[boxx][boxy] = False
                    elif hasWon(revealedBoxes): # check if all pairs found
                        gameWonAnimation(mainBoard)
                        pygame.time.wait(2000)

                        # Reset the board
                        mainBoard = getRandomizedBoard()
                        revealedBoxes = generateRevealedBoxesData(False)

                        # Show the fully unrevealed board for a second.
                        drawBoard(mainBoard, revealedBoxes)
                        pygame.display.update()
                        pygame.time.wait(1000)

                        # Replay the start game animation.
                        startGameAnimation(mainBoard)
                    firstSelection = None # reset firstSelection variable

        # Redraw the screen and wait a clock tick.
        pygame.display.update()
        FPSCLOCK.tick(FPS)


def generateRevealedBoxesData(val):
    revealedBoxes = []
    for i in range(BOARDWIDTH):
        revealedBoxes.append([val] * BOARDHEIGHT)
    return revealedBoxes


def getRandomizedBoard():
    # Get a list of every possible shape in every possible color.
    icons = []
    for color in ALLCOLORS:
        for shape in ALLSHAPES:
            icons.append( (shape, color) )

    random.shuffle(icons) # randomize the order of the icons list
    numIconsUsed = int(BOARDWIDTH * BOARDHEIGHT / 2) # calculate how many icons are needed
    icons = icons[:numIconsUsed] * 2 # make two of each
    random.shuffle(icons)

    # Create the board data structure, with randomly placed icons.
    board = []
    for x in range(BOARDWIDTH):
        column = []
        for y in range(BOARDHEIGHT):
            column.append(icons[0])
            del icons[0] # remove the icons as we assign them
        board.append(column)
    return board


def splitIntoGroupsOf(groupSize, theList):
    # splits a list into a list of lists, where the inner lists have at
    # most groupSize number of items.
    result = []
    for i in range(0, len(theList), groupSize):
        result.append(theList[i:i + groupSize])
    return result


def leftTopCoordsOfBox(boxx, boxy):
    # Convert board coordinates to pixel coordinates
    left = boxx * (BOXSIZE + GAPSIZE) + XMARGIN
    top = boxy * (BOXSIZE + GAPSIZE) + YMARGIN
    return (left, top)


def getBoxAtPixel(x, y):
    for boxx in range(BOARDWIDTH):
        for boxy in range(BOARDHEIGHT):
            left, top = leftTopCoordsOfBox(boxx, boxy)
            boxRect = pygame.Rect(left, top, BOXSIZE, BOXSIZE)
            if boxRect.collidepoint(x, y):
                return (boxx, boxy)
    return (None, None)


def drawIcon(shape, color, boxx, boxy):
    quarter = int(BOXSIZE * 0.25) # syntactic sugar
    half =    int(BOXSIZE * 0.5)  # syntactic sugar

    left, top = leftTopCoordsOfBox(boxx, boxy) # get pixel coords from board coords
    # Draw the shapes
    if shape == DONUT:
        pygame.draw.circle(DISPLAYSURF, color, (left + half, top + half), half - 5)
        pygame.draw.circle(DISPLAYSURF, BGCOLOR, (left + half, top + half), quarter - 5)
    elif shape == SQUARE:
        pygame.draw.rect(DISPLAYSURF, color, (left + quarter, top + quarter, BOXSIZE - half, BOXSIZE - half))
    elif shape == DIAMOND:
        pygame.draw.polygon(DISPLAYSURF, color, ((left + half, top), (left + BOXSIZE - 1, top + half), (left + half, top + BOXSIZE - 1), (left, top + half)))
    elif shape == LINES:
        for i in range(0, BOXSIZE, 4):
            pygame.draw.line(DISPLAYSURF, color, (left, top + i), (left + i, top))
            pygame.draw.line(DISPLAYSURF, color, (left + i, top + BOXSIZE - 1), (left + BOXSIZE - 1, top + i))
    elif shape == OVAL:
        pygame.draw.ellipse(DISPLAYSURF, color, (left, top + quarter, BOXSIZE, half))


def getShapeAndColor(board, boxx, boxy):
    # shape value for x, y spot is stored in board[x][y][0]
    # color value for x, y spot is stored in board[x][y][1]
    return board[boxx][boxy][0], board[boxx][boxy][1]


def drawBoxCovers(board, boxes, coverage):
    # Draws boxes being covered/revealed. "boxes" is a list
    # of two-item lists, which have the x & y spot of the box.
    for box in boxes:
        left, top = leftTopCoordsOfBox(box[0], box[1])
        pygame.draw.rect(DISPLAYSURF, BGCOLOR, (left, top, BOXSIZE, BOXSIZE))
        shape, color = getShapeAndColor(board, box[0], box[1])
        drawIcon(shape, color, box[0], box[1])
        if coverage > 0: # only draw the cover if there is an coverage
            pygame.draw.rect(DISPLAYSURF, BOXCOLOR, (left, top, coverage, BOXSIZE))
    pygame.display.update()
    FPSCLOCK.tick(FPS)


def revealBoxesAnimation(board, boxesToReveal):
    # Do the "box reveal" animation.
    for coverage in range(BOXSIZE, (-REVEALSPEED) - 1, -REVEALSPEED):
        drawBoxCovers(board, boxesToReveal, coverage)


def coverBoxesAnimation(board, boxesToCover):
    # Do the "box cover" animation.
    for coverage in range(0, BOXSIZE + REVEALSPEED, REVEALSPEED):
        drawBoxCovers(board, boxesToCover, coverage)


def drawBoard(board, revealed):
    # Draws all of the boxes in their covered or revealed state.
    for boxx in range(BOARDWIDTH):
        for boxy in range(BOARDHEIGHT):
            left, top = leftTopCoordsOfBox(boxx, boxy)
            if not revealed[boxx][boxy]:
                # Draw a covered box.
                pygame.draw.rect(DISPLAYSURF, BOXCOLOR, (left, top, BOXSIZE, BOXSIZE))
            else:
                # Draw the (revealed) icon.
                shape, color = getShapeAndColor(board, boxx, boxy)
                drawIcon(shape, color, boxx, boxy)


def drawHighlightBox(boxx, boxy):
    left, top = leftTopCoordsOfBox(boxx, boxy)
    pygame.draw.rect(DISPLAYSURF, HIGHLIGHTCOLOR, (left - 5, top - 5, BOXSIZE + 10, BOXSIZE + 10), 4)


def startGameAnimation(board):
    # Randomly reveal the boxes 8 at a time.
    coveredBoxes = generateRevealedBoxesData(False)
    boxes = []
    for x in range(BOARDWIDTH):
        for y in range(BOARDHEIGHT):
            boxes.append( (x, y) )
    random.shuffle(boxes)
    boxGroups = splitIntoGroupsOf(8, boxes)

    drawBoard(board, coveredBoxes)
    for boxGroup in boxGroups:
        revealBoxesAnimation(board, boxGroup)
        coverBoxesAnimation(board, boxGroup)


def gameWonAnimation(board):
    # flash the background color when the player has won
    coveredBoxes = generateRevealedBoxesData(True)
    color1 = LIGHTBGCOLOR
    color2 = BGCOLOR

    for i in range(13):
        color1, color2 = color2, color1 # swap colors
        DISPLAYSURF.fill(color1)
        drawBoard(board, coveredBoxes)
        pygame.display.update()
        pygame.time.wait(300)


def hasWon(revealedBoxes):
    # Returns True if all the boxes have been revealed, otherwise False
    for i in revealedBoxes:
        if False in i:
            return False # return False if any boxes are covered.
    return True


if __name__ == '__main__':
    main()




pygame 2.1.2 (SDL 2.0.18, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

D:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### 打磚塊
來源: https://gist.github.com/hank08tw/4212ced7d404adc073acc9c290a5b6f7

In [20]:
#v2
import pygame, random, math, time

class ball(pygame.sprite.Sprite):
    direction = 0
    speed = 0
    x = 0
    y = 0
    dx = 0
    dy = 0

    def __init__(self, inputspeed, inputx, inputy, radius, color):
        pygame.sprite.Sprite.__init__(self)
        self.speed = inputspeed
        self.x = inputx
        self.y = inputy
        self.image = pygame.Surface([radius * 2, radius * 2])
        self.image.fill((255, 255, 255))
        pygame.draw.circle(self.image, color, (radius, radius), radius, 0)
        self.rect = self.image.get_rect()
        self.rect.center = (inputx, inputy)
        self.direction = random.randint(20, 70)

    def update(self):
        radian = math.radians(self.direction)
        self.dx = self.speed * math.cos(radian)
        self.dy = -self.speed * math.sin(radian)
        self.x += self.dx
        self.y += self.dy
        self.rect.x = self.x
        self.rect.y = self.y
        if (self.rect.left <= 0 or self.rect.right >= window.get_width()):
            self.hitleftright()
        elif (self.rect.top <= 0):
            self.hittop()

    def hittop(self):
        self.direction = 360 - self.direction

    def hitleftright(self):
        if 180 >= self.direction >= 0:
            self.direction = 180 - self.direction
        else:
            self.direction = 540 - self.direction

class brick(pygame.sprite.Sprite):
    def __init__(self, color, x, y):
        pygame.sprite.Sprite.__init__(self)
        self.image = pygame.Surface([40, 15])
        self.image.fill(color)
        self.rect = self.image.get_rect()
        self.rect.x = x
        self.rect.y = y

class moveclip(pygame.sprite.Sprite):
    def __init__(self, color, x, y):
        pygame.sprite.Sprite.__init__(self)
        self.image = pygame.Surface([50, 20])
        self.image.fill(color)
        self.rect = self.image.get_rect()
        self.rect.x = x
        self.rect.y = y

    def update(self):
        pos = pygame.mouse.get_pos()
        self.rect.x = pos[0]
        if self.rect.x > window.get_width() - self.rect.width:
            self.rect.x = window.get_width() - self.rect.width

pygame.init()
font = pygame.font.SysFont("SimHei", 20)

def gameover(message):
    global run
    text = font.render(message, 1, (255, 0, 0))
    window.blit(text, (window.get_width() / 2 - 150, window.get_height() / 2))
    pygame.display.update()
    run = False
    time.sleep(3)

point = 0
window = pygame.display.set_mode((600, 400))
pygame.display.set_caption("breakingbricks")
background = pygame.Surface(window.get_size())
background = background.convert()
background.fill((255, 255, 255))
window.blit(background, (0, 0))
allsprite = pygame.sprite.Group()
bricksprite = pygame.sprite.Group()
controllersprite = pygame.sprite.Group()
theball = ball(6, 100, 100, 20, (0, 0, 255))
allsprite.add(theball)
controller = moveclip((255, 0, 0), 0, 350)
allsprite.add(controller)
controllersprite.add(controller)
clock = pygame.time.Clock()

for i in range(3):
    for j in range(0, 15):
        thebrick = brick((random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)), j * 40 + 1, i * 15 + 1)
        bricksprite.add(thebrick)
        allsprite.add(thebrick)
msgstart = "click left button on the mouse to start the game"
msgstartdisplay = font.render(msgstart, 100, (255, 0, 0))
window.blit(msgstartdisplay, (window.get_width() / 2 - 50, window.get_height() / 2))
playing = False
run = True

while run:
    clock.tick(40)
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            run = False
    button = pygame.mouse.get_pressed()

    if button[0]:
        playing = True

    if playing:
        window.blit(background, (0, 0))

        if theball.y >= window.get_height():
            gameover("youlose!")

        controller.update()
        hitbrick = pygame.sprite.spritecollide(theball, bricksprite, True)

        if len(hitbrick) > 0:
            point += len(hitbrick)
            theball.rect.y += 20
            theball.hittop()
            if len(bricksprite) == 0:
                gameover("youwin!")

        hitclip = pygame.sprite.spritecollide(theball, controllersprite, False)

        if len(hitclip) > 0:
            theball.hittop()

        theball.update()

    allsprite.draw(window)

    msgscore = "score: " + str(point)
    msgscoredisplay = font.render(msgscore, 5, (255, 0, 0))
    window.blit(msgscoredisplay, (window.get_width() - 60, window.get_height() - 30))
    pygame.display.update()

pygame.quit()


### 以下提供幾個半成品遊戲，大家可以挑一個將其完成、擴增，也可以自己設計新的遊戲
或是直接要chatgpt生成一個，再將其改成可以正常遊戲~

### 消消樂
1.將這個遊戲修好，並且 新增音效、換顏色等等<br>
2. 進階挑戰，要碰撞的方塊是撞完一個再隨機生成的<br>
或是3. 新增自己的規則!

In [12]:
import pygame
import sys
import random
import time

# 初始化Pygame
pygame.init()

# 游戏窗口设置
window_width = 400
window_height = 400
window = pygame.display.set_mode((window_width, window_height))
pygame.display.set_caption("碰撞遊戲")

# 颜色设置
white = (255, 255, 255)
blue = (0, 0, 255)
black = (0, 0, 0)
font = pygame.font.Font(None, 36)  # 创建一个字体对象

# 正方形设置
square_size = 30
player_size = 30

# 初始化玩家和目标位置
player_x = window_width // 2
player_y = window_height - player_size
square_a_x = random.randint(0, window_width - square_size)
square_a_y = random.randint(0, window_height - square_size)
square_b_x = random.randint(0, window_width - square_size)
square_b_y = random.randint(0, window_height - square_size)

# 计时器设置
start_time = time.time()  # 初始化开始时间

def draw_square(x, y):
    pygame.draw.rect(window, blue, (x, y, square_size, square_size))
    
def clear_square(x, y):
    pygame.draw.rect(window, black, (x, y, square_size, square_size))

def draw_player(x, y):
    pygame.draw.rect(window, white, (x, y, player_size, player_size))

def check_collision(x1, y1, x2, y2, size):
    if x1 < x2 + size and x1 + player_size > x2 and y1 < y2 + size and y1 + player_size > y2:
        return True
    return False

running = True
score = 0
hit_square_a = False
hit_square_b = False
move_pending = False

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            move_pending = True

    keys = pygame.key.get_pressed()
    if move_pending:
        if keys[pygame.K_LEFT] and player_x > 0:
            player_x -= 15
            move_pending = False
        elif keys[pygame.K_RIGHT] and player_x < window_width - player_size:
            player_x += 15
            move_pending = False
        elif keys[pygame.K_UP] and player_y > 0:
            player_y -= 15
            move_pending = False
        elif keys[pygame.K_DOWN] and player_y < window_height - player_size:
            player_y += 15
            move_pending = False

    window.fill((0, 0, 0))

    if not hit_square_a:
        draw_square(square_a_x, square_a_y)
    if not hit_square_b:
        draw_square(square_b_x, square_b_y)

    if check_collision(player_x, player_y, square_a_x, square_a_y, square_size):
        hit_square_a = True
        
    if check_collision(player_x, player_y, square_b_x, square_b_y, square_size):
        hit_square_b = True
        
    if hit_square_a and hit_square_b:
        running = False
        
    else:
        draw_player(player_x, player_y)

        # 显示分数
        if start_time is not None:
            end_time = time.time()
            score = int(1000-(end_time - start_time)*10)
            text = font.render(f"Final Score: {score}", True, white)
            window.blit(text, (window_width // 2 - 100, window_height - 18))

        pygame.display.update()

# 游戏结束，等待用户退出
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()


SystemExit: 

## 貪吃蛇
1. 試試看將蛇的食物換成自己準備的圖片<br>
2. 蛇如果踩到特定方塊，蛇蛇變兩倍寬(?#

In [17]:
import pygame
import sys
import random

# 初始化游戏
pygame.init()

# 游戏参数
WIDTH, HEIGHT = 700, 700
GRID_SIZE = 35
GRID_WIDTH = WIDTH // GRID_SIZE
GRID_HEIGHT = HEIGHT // GRID_SIZE

# 颜色
WHITE = (255, 255, 255)
GREEN = (0, 255, 0)
RED = (255, 0, 0)

# 初始化屏幕
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("贪吃蛇游戏")

# 初始化蛇
snake = [(5, 5)]
snake_direction = (1, 0)

# 初始化食物
food = (random.randint(0, GRID_WIDTH - 1), random.randint(0, GRID_HEIGHT - 1))

# 游戏循环
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_UP and snake_direction != (0, 1):
                snake_direction = (0, -1)
            elif event.key == pygame.K_DOWN and snake_direction != (0, -1):
                snake_direction = (0, 1)
            elif event.key == pygame.K_LEFT and snake_direction != (1, 0):
                snake_direction = (-1, 0)
            elif event.key == pygame.K_RIGHT and snake_direction != (-1, 0):
                snake_direction = (1, 0)

    # 移动蛇
    x, y = snake[0]
    new_head = (x + snake_direction[0], y + snake_direction[1])
    snake.insert(0, new_head)

    # 检查是否吃到食物
    if snake[0] == food:
        food = (random.randint(0, GRID_WIDTH - 1), random.randint(0, GRID_HEIGHT - 1))
    else:
        snake.pop()

    # 检查游戏结束
    if (
        snake[0][0] < 0
        or snake[0][0] >= GRID_WIDTH
        or snake[0][1] < 0
        or snake[0][1] >= GRID_HEIGHT
        or snake[0] in snake[1:]
    ):
    # 绘制屏幕
        screen.fill(WHITE)

    for segment in snake:
        pygame.draw.rect(
            screen, GREEN, (segment[0] * GRID_SIZE, segment[1] * GRID_SIZE, GRID_SIZE, GRID_SIZE)
        )

    pygame.draw.rect(screen, RED, (food[0] * GRID_SIZE, food[1] * GRID_SIZE, GRID_SIZE, GRID_SIZE))

    pygame.display.update()
    pygame.time.delay(100)


    pygame.quit()
    sys.exit()


SystemExit: 

### 打地鼠
1. 把地鼠改成真的地鼠圖片<br>
2. 限時30秒 比分數<br>
3. 加聲音<br>
4. 自己加新規則~<br>
5. 用鎚子打

In [11]:
import pygame
import sys
import random
import time

# 初始化游戏
pygame.init()

# 游戏参数
WIDTH, HEIGHT = 800, 600
MOLE_SIZE = 50
MOLE_SPEED = 2
SCORE = 0

# 颜色
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

# 初始化屏幕
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("打地鼠")

# 地鼠
mole_color = (139, 69, 19)  # 棕色
mole_rect = pygame.Rect(0, 0, MOLE_SIZE, MOLE_SIZE)
mole_x = random.randint(0, WIDTH - MOLE_SIZE)
mole_y = random.randint(0, HEIGHT - MOLE_SIZE)
mole_img = pygame.image.load('地鼠.png')  # Replace 'custom_mole.png' with the path to your image
mole_img = pygame.transform.scale(mole_img, (MOLE_SIZE, MOLE_SIZE))  # Resize the image to match the mole's siz
mole_visible = False

# 初始化字体
font = pygame.font.Font(None, 36)
start_time=time.time()
running = True
# 游戏循循环
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

        if event.type == pygame.MOUSEBUTTONDOWN:
            if mole_visible and mole_rect.collidepoint(event.pos):
                SCORE += 1
                mole_visible = False
    # 移动地鼠
    if not mole_visible:
        mole_x = random.randint(0, WIDTH - MOLE_SIZE)
        mole_y = random.randint(0, HEIGHT - MOLE_SIZE)
        mole_rect.topleft = (mole_x, mole_y)  # 更新地鼠矩形的位置
        mole_visible = True

    # 清空屏幕
    screen.fill(WHITE)

    # 绘制地鼠
    if mole_visible:
        screen.blit(mole_img, mole_rect.topleft)

    end_time=time.time()
    # 計時器
    game_time = 30-(end_time-start_time)  # 30秒計時器
    font_1 = pygame.font.Font(None, 36) 
    text_1 = font_1.render(f"game_time: {game_time:.0f}", True, BLACK)
    text_rect_1 = text_1.get_rect(topleft=(10, 30))
    screen.blit(text_1, text_rect_1)
    if (game_time<=0):
        running= False
    # 绘制分数
    text = font.render(f"Score: {SCORE}", True, BLACK)
    text_rect = text.get_rect(topleft=(10, 10))
    screen.blit(text, text_rect)

    pygame.display.update()


FileNotFoundError: No file 'mole_appear.wav' found in working directory 'C:\Users\MAGGIE\jupyter notebook'.

### 轉盤選餐廳
1. 加扇形的框框線
2. 客製輸入餐廳名稱

In [9]:
import pygame
import sys
import random
import math

pygame.init()
screen = pygame.display.set_mode((400, 400))
pygame.display.set_caption('選餐廳')

# 定义轮盘参数和扇形
wheel_radius = 150
num_restaurants = 6
angle_per_sector = 360 / num_restaurants
restaurant_names = ["Restaurant 1", "Restaurant 2", "Restaurant 3", "Restaurant 4", "Restaurant 5", "Restaurant 6"]
selected_restaurant = None

# 定义指针参数
pointer_length = 20
pointer_angle = 0

# 定义按钮参数
button_rect = pygame.Rect(160, 350, 80, 30)
button_font = pygame.font.Font(None, 36)
restaurant_font = pygame.font.Font(None, 24)

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.MOUSEBUTTONDOWN and button_rect.collidepoint(event.pos):
            # 用户点击了停止按钮
            selected_restaurant = random.choice(restaurant_names)

    if selected_restaurant is None:
        # 如果没有选择餐厅，继续旋转轮盘
        pointer_angle += 5  # 调整旋转速度

    screen.fill((255, 255, 255))

    # 绘制轮盘
    for i in range(num_restaurants):
        start_angle = math.radians(i * angle_per_sector)
        end_angle = math.radians((i + 1) * angle_per_sector)
        color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))  # 随机颜色
        pygame.draw.arc(screen, color, (125, 125, wheel_radius * 2, wheel_radius * 2), start_angle, end_angle, 0)

    # 绘制指针
    pointer_x = 200 + (wheel_radius - pointer_length) * math.cos(math.radians(pointer_angle))
    pointer_y = 200 + (wheel_radius - pointer_length) * math.sin(math.radians(pointer_angle))
    pygame.draw.line(screen, (255, 0, 0), (200, 200), (pointer_x, pointer_y), 5)

    # 绘制按钮
    pygame.draw.rect(screen, (0, 0, 255), button_rect)
    button_text = button_font.render("stop", True, (255, 255, 255))
    screen.blit(button_text, (175, 355))

    # 绘制餐厅名称
    for i in range(num_restaurants):
        text = restaurant_font.render(restaurant_names[i], True, (0, 0, 0))
        text_rect = text.get_rect()
        text_rect.center = (200 + (wheel_radius - 20) * math.cos(math.radians(i * angle_per_sector + angle_per_sector / 2)),
                            200 + (wheel_radius - 20) * math.sin(math.radians(i * angle_per_sector + angle_per_sector / 2)))
        screen.blit(text, text_rect)

    pygame.display.flip()

# 游戏循环结束后，显示选择的餐厅
if selected_restaurant:
    print(f"選擇的是：{selected_restaurant}")

pygame.quit()
sys.exit()


選擇的是：Restaurant 4


SystemExit: 

#### 其他可以嘗試的
#### 模擬鋼琴、戳戳樂、剪刀石頭布

##### 模擬鋼琴 - 畫長方形當鍵盤，當偵測到使用者按到該鍵盤，就播該鍵盤代表的音效。